In [8]:
import pandas as pd
import joblib
import numpy as np
import PySimpleGUI as sg
import matplotlib.pyplot as plt
import PySimpleGUI as sg
import numpy as np
import pandas as pd
from pickle import load
from PIL import Image



# === GUI Color Scheme ===
text_color = 'black'
highlight_color = 'navy'
frame_color = '#f0f0f0'     # light gray for frames
bg_color = '#e6ecf0'        # soft background for window
output_color = ('black', '#c7e9c0')  # black text on light green background
button_color = ('white', '#4682B4')  # white on steel blue


# === GUI Design Settings ===
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'mathtext.fontset': 'cm',
    'axes.labelsize': 12,
    'axes.titlesize': 12,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12
})

# === Input Ranges and Mappings ===
fiber_map = {'Yes': 1, 'No': 2}
fiber_choices = list(fiber_map.keys())

ranges = {
    'D': (50, 150),
    'h': (100, 300),
    'fco': (74.52, 166.95),
    'Ef': (16.56, 336),
    'ff': (389.7, 5780.25),
    'tf': (0.1503, 5.754),
    'εhr': (0.423, 2.9295)
}

# === GUI Appearance Settings ===
sg.theme('LightGrey1')
t, td, td2 = 32, 35, 8

# === Input Layout ===
input_frame = [
    [sg.Text('Fiber:', size=(t, 1)), sg.Combo(fiber_choices, default_value='Yes', key='-f1-', size=(td2, 1))],
    [sg.Text('Concrete core diameter, D (mm)', size=(t, 1), text_color=text_color), sg.InputText(key='-f2-', size=(td2, 1))],
    [sg.Text('Specimen height, h (mm)', size=(t, 1), text_color=text_color), sg.InputText(key='-f3-', size=(td2, 1))],
    [sg.Text('unconfined strength of UHPC, fco (MPa)', size=(t, 1), text_color=text_color), sg.InputText(key='-f4-', size=(td2, 1))],
    [sg.Text('Elastic modulus, Ef (GPa)', size=(t, 1), text_color=text_color), sg.InputText(key='-f5-', size=(td2, 1))],
    [sg.Text('Tensile strength ff (MPa)', size=(t, 1), text_color=text_color), sg.InputText(key='-f6-', size=(td2, 1))],
    [sg.Text('FRP thickness, tf (mm)', size=(t, 1), text_color=text_color), sg.InputText(key='-f7-', size=(td2, 1))],
    [sg.Text('Actual hoop strain, εhr (%)', size=(t, 1), text_color=text_color), sg.InputText(key='-f8-', size=(td2, 1))],
]

range_frame = [[sg.Text('Fiber: Yes or No')]] + [
    [sg.Text(f"{k}: {v[0]} to {v[1]}")] for k, v in ranges.items()
]

layout = [
    [sg.Text('Developed by IA Tijani & C Jiang', font=('Times New Roman', 8))],
    [sg.Text('The University of British Columbia & Western Sydney University', font=('Times New Roman', 8))],

    [sg.Column([[sg.Frame('Input Parameters', input_frame, background_color=frame_color)]], background_color=bg_color),
     sg.Column([[sg.Frame('Model Input Range', range_frame, background_color=frame_color)]], background_color=bg_color)],

    [sg.Frame('Prediction Output', [[sg.Text('Predicted fcc (MPa):', size=(25, 1), text_color=text_color),
                                     sg.InputText(key='-OP-', size=(td2 + 5, 1), text_color=output_color[0], background_color=output_color[1], disabled=True)]], background_color=frame_color)],

    [sg.Button('Predict', button_color=button_color), sg.Button('Cancel', button_color=('white', '#B22222'))]
]

# === Run GUI ===
window = sg.Window('FRP-conUHPC-StackedHybridOptML', layout, background_color=bg_color)

# Load trained model
model = load(open('prediction_model_for_GUI.pkl', 'rb'))

while True:
    event, values = window.read()
    if event in (sg.WIN_CLOSED, 'Cancel'):
        break

    if event == 'Predict':
        try:
            fiber = fiber_map[values['-f1-']]
            D = float(values['-f2-'])
            h = float(values['-f3-'])
            fco = float(values['-f4-'])
            Ef = float(values['-f5-'])
            ff = float(values['-f6-'])
            tf = float(values['-f7-'])
            εhr = float(values['-f8-'])

            # Range Checks
            if not (ranges['D'][0] <= D <= ranges['D'][1]):
                sg.popup("D must be between 50 and 150 mm.")
                continue
            if not (ranges['h'][0] <= h <= ranges['h'][1]):
                sg.popup("h must be between 100 and 300 mm.")
                continue
            if not (ranges['fco'][0] <= fco <= ranges['fco'][1]):
                sg.popup("fco must be between 74.52 and 166.95 MPa.")
                continue
            if not (ranges['Ef'][0] <= Ef <= ranges['Ef'][1]):
                sg.popup("Ef must be between 16.56 and 336 GPa.")
                continue
            if not (ranges['ff'][0] <= ff <= ranges['ff'][1]):
                sg.popup("ff must be between 389.7 and 5780.25 MPa.")
                continue
            if not (ranges['tf'][0] <= tf <= ranges['tf'][1]):
                sg.popup("tf must be between 0.1503 and 5.754 mm.")
                continue
            if not (ranges['εhr'][0] <= εhr <= ranges['εhr'][1]):
                sg.popup("εhr must be between 0.423 and 2.9295 %.")
                continue

            # Predict using joblib
            input_array = np.array([[fiber, D, h, fco, Ef, ff, tf, εhr]])
            prediction = model.predict(input_array)[0]

            window['-OP-'].update(f"{prediction:.2f} MPa")

        except ValueError:
            sg.popup("⚠️ Please enter valid numeric values.")

window.close()
